In [1]:
import os, sys
sys.path.append('../')
os.chdir('../')

In [10]:
import torch
import random
import numpy as np
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import BertForSequenceClassification
from src.utils import set_seed

In [11]:
# Fungsi untuk memuat model yang sudah dilatih
def load_model(model, load_model_name="model", load_model_dir="E:/code/project-list/bert-hfacs/models/model_trained/"):
    # Pastikan direktori ada, jika tidak buat
    os.makedirs(load_model_dir, exist_ok=True)

    # Gabungkan nama model dengan ekstensi .pth
    load_model_name_with_extension = load_model_name + ".pth"

    # Gabungkan direktori dan nama file model
    load_path = os.path.join(load_model_dir, load_model_name_with_extension)

    # Memuat model state_dict
    model.load_state_dict(
        torch.load(
            load_path,
            map_location=torch.device('cpu'),  # Ganti ke 'cuda' jika menggunakan GPU
        )
    )
    print("Model Weight Loaded")
    return model


In [14]:
model_path = "E:/code/project-list/bert-hfacs/models/indobert_base"

In [ ]:
# 1. Buat config dengan arsitektur paling sederhana
config = BertConfig.from_pretrained(
    model_path,
    hidden_size=768,  # Ukuran hidden layer, harus sama denan intermediate size
    num_attention_heads=12,  # 1 attention head
    num_hidden_layers=12,
    num_labels= 2
)

TypeError: PretrainedConfig.from_pretrained() missing 1 required positional argument: 'pretrained_model_name_or_path'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_path)

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    model_path,
    config=config,
)

RuntimeError: Error(s) in loading state_dict for BertForSequenceClassification:
	size mismatch for embeddings.word_embeddings.weight: copying a param with shape torch.Size([50000, 768]) from checkpoint, the shape in current model is torch.Size([30522, 768]).
	You may consider adding `ignore_mismatched_sizes=True` in the model `from_pretrained` method.

In [ ]:
model = load_model(model, "indobert_base_E10_LR1e-2_BS8")

In [ ]:
model = model.cuda()

In [ ]:
# 2. Contoh input teks
input_text = "supir mobil kantuk"

In [6]:
subwords = tokenizer.encode(input_text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

In [7]:
logits = model(subwords)[0]

In [8]:
labels = torch.tensor([1]) 